[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Advanced Vision-Based ADAS with Real-Time Schematic Mapping**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## Project Overview
It is an advanced driver-assistance system (ADAS) that leverages the cutting-edge **YOLO11x** instance segmentation architecture to transform raw 4K video feeds into actionable driving intelligence. Unlike standard object detection projects, it processes environment data into a **Digital Twin Schematic**, providing a top-down radar view and real-time directional steering guidance.

The system performs three core functions simultaneously:
1.  **Multi-Class Semantic Segmentation:** High-fidelity masking of vehicles (Cars/Trucks) and road infrastructure (Lane Lines).
2.  **Dynamic Perimeter Mapping:** A custom-mapped 8-point sensor zone that monitors road curvature and lane drift.
3.  **Schematic Radar HUD:** A noise-filtered, top-down visualization that maps detections onto a digital canvas, mirroring professional-grade autonomous vehicle dashboards.

---

## Technical Implementation
* **Core Model:** YOLO11x (Extra Large) Instance Segmentation.
* **Inference Engine:** Optimized for NVIDIA T4/P100 GPUs using FP16 half-precision and 1024px image scaling.
* **Logic Layer:** * **Perspective-Aware Scaling:** Maps 2D image coordinates to a proportional 600x900 radar schematic.
    * **Centroid Directional Analysis:** Calculates the geometric center of the sensor zone to determine "Turn Left" vs "Turn Right" signals based on lane line intersection.
    * **Noise Filtering:** Uses a black-canvas rendering technique to exclude non-relevant environmental objects, ensuring a clean HUD.

---

## Real-World Applications



### 1. Level 2+ Autonomous Driving
The system acts as a "second set of eyes," providing lane-keeping assistance (LKA). By detecting when a lane line enters the custom perimeter, the system can trigger automated steering corrections or haptic driver alerts.

### 2. Digital Twin Dashboards
The schematic radar view can be integrated into high-end vehicle clusters (like those in Tesla, Rivian, or Lucid), allowing drivers to visualize the "AI's mind" and understand how the vehicle perceives its surroundings in low-visibility conditions.

### 3. Smart Infrastructure & Traffic Management
Beyond in-vehicle use, the logic can be applied to static 5G smart-city cameras to monitor traffic flow, detect lane violations, and predict potential collisions at complex intersections.

### 4. Blind Spot & Collision Avoidance
The perimeter mapping logic can be extended to the sides and rear of the vehicle to create a 360-degree safety bubble, identifying "shadow" vehicles in the radar view that might be missed by traditional mirrors.

##  Installation & Setup

To use the **Labellerr Fine-Tune Utilities** in your environment (Kaggle, Colab, or Local), you can clone the repository directly into your workspace. This allows you to access the specialized conversion scripts for your ADAS dataset.

In [2]:
!git clone https://github.com/Labellerr/yolo_finetune_utils.git

Cloning into 'yolo_finetune_utils'...


## Random Frame Extraction from Video

Extracts a fixed number of high-quality frames from one or more videos to create an image dataset for annotation and training.

### 🔹 Purpose
- Convert raw manufacturing videos into individual image frames  
- Perform random sampling to avoid frame bias  
- Prepare data for annotation and YOLO training  


In [3]:
from yolo_finetune_utils.frame_extractor import extract_random_frames

extract_random_frames(
    paths=['5382495-uhd_3840_2160_24fps.mp4'],
    total_images=50,
    out_dir="input_data",
    jpg_quality=100,
    seed=42
)

[✓] Extracted 50 frames to folder: input_data


## Download Annotations from Labellerr

After completing data labeling on the **Labellerr** platform, export the annotations in **COCO JSON format**.

Download the COCO JSON file from the Labellerr website and upload it into this project workspace to use it for further dataset preparation and training.

This COCO JSON file will be used in the next steps for:
- Frame–annotation alignment
- COCO → YOLO format conversion
- Model training and evaluation


# COCO to YOLO Format Conversion

Converts COCO-style segmentation annotations to YOLO segmentation dataset format.  
- Requires: `annotation.json` and images in `frames_output` directory.
- Output: Generated YOLO dataset folder.
- Parameters: allows train/val split, shuffling, and verbose mode.


In [2]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

coco_to_yolo_converter(
    json_path="export-#73b58c11-8ef2-4e55-9e7e-dc1190179575.json",
    images_dir="input_data",
    output_dir="yolo_dataset",
    use_split=True,
    train_ratio=0.7,
    val_ratio=0.2,
    test_ratio=0.1,
    shuffle=True,
    verbose=True
)


Conversion complete. Stats: {'train': 22, 'val': 6, 'test': 4}


{'stats': {'train': 22, 'val': 6, 'test': 4}, 'output_dir': 'yolo_dataset'}

## Model Training Configuration

This stage of the pipeline initializes the fine-tuning process for the **YOLO11x-seg** model. By leveraging the **Extra Large (x)** variant, we prioritize maximum detection accuracy and segmentation mask smoothness, which are critical for safety-critical ADAS applications.

In [ ]:
from ultralytics import YOLO

# Load the nano segmentation model
model = YOLO('yolo11x-seg.pt')

# Start training
model.train(
    data='/content/adas_data/yolo_dataset/data.yaml',
    epochs=150,
    imgsz=640,
    device=0, 
    project='/content/drive/MyDrive/ADAS_Project',
    name='adas_final_run'
)

## Interactive Perimeter Mapping Tool

The **Perimeter Mapper** is a preprocessing utility designed to extract precise coordinate arrays for custom polygons. In this project, it is used to define the "sensor zone" on the road. By sampling a frame at **5.5 seconds**, we ensure the road geometry is fully visible before mapping. Essential for Lane Departure Warning Systems (LDWS). It acts as a trigger: when lane masks intersect this zone, the system calculates steering adjustments.

In [11]:
import cv2
import numpy as np

# Global list to store polygon vertices
pts = []

def draw_polygon(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        pts.append([x, y])
        print(f"Point {len(pts)} added: {x, y}")
    elif event == cv2.EVENT_RBUTTONDOWN:
        if len(pts) > 0:
            pts.pop() # Remove last point on right click
            print("Last point removed.")

# 1. Load your specific ADAS video
video_path = r'D:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\5382495-uhd_3840_2160_24fps.mp4'
cap = cv2.VideoCapture(video_path)

# Get FPS
fps = cap.get(cv2.CAP_PROP_FPS)

# Calculate frame number at 3 seconds
frame_number = int(fps * 5.5)

# Set video position to that frame
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

# Read frame
ret, frame = cap.read()

# Resize for your MSI screen (Mapping on 1280x720)
scale_w, scale_h = 1280, 720
frame_reshaped = cv2.resize(frame, (scale_w, scale_h))

cv2.namedWindow('Polygon Mapper')
cv2.setMouseCallback('Polygon Mapper', draw_polygon)

print("--- INSTRUCTIONS ---")
print("1. Left Click to add points (make as many as you want for a curve)")
print("2. Right Click to undo a point")
print("3. Press 's' to print the final array")
print("4. Press 'q' to quit")

while True:
    img_display = frame_reshaped.copy()
    
    # Draw the lines connecting your points
    if len(pts) > 1:
        cv2.polylines(img_display, [np.array(pts)], False, (0, 255, 255), 2)
    
    # Draw a small circle at each vertex
    for p in pts:
        cv2.circle(img_display, (p[0], p[1]), 4, (0, 0, 255), -1)
    
    cv2.imshow('Polygon Mapper', img_display)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):
        print("\n--- COPY THIS INTO YOUR MAIN SCRIPT ---")
        print(f"ZONE_POINTS = np.array({pts}, np.int32)")
        # Show closed polygon preview
        if len(pts) > 2:
            cv2.fillPoly(img_display, [np.array(pts)], (0, 255, 0))
            cv2.imshow('Polygon Mapper', img_display)
            cv2.waitKey(1000)
            
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

--- INSTRUCTIONS ---
1. Left Click to add points (make as many as you want for a curve)
2. Right Click to undo a point
3. Press 's' to print the final array
4. Press 'q' to quit
Point 1 added: (428, 428)
Point 2 added: (535, 272)
Point 3 added: (716, 257)
Point 4 added: (845, 403)
Point 5 added: (692, 393)
Point 6 added: (552, 401)
Point 7 added: (428, 425)

--- COPY THIS INTO YOUR MAIN SCRIPT ---
ZONE_POINTS = np.array([[428, 428], [535, 272], [716, 257], [845, 403], [692, 393], [552, 401], [428, 425]], np.int32)
Point 8 added: (620, 329)


Here is the clean Markdown code for the ADAS High-Visibility Signal System. This version is concise, structured, and ready to be added to your project documentation.

Markdown
# ADAS High-Visibility Signal System

The **Signal System** is the decision-making core of this ADAS project. It processes real-time segmentation masks to provide directional steering guidance and a high-contrast HUD for the driver.



---

## Technical Logic

* **Model:** YOLO11x-seg (Inference at 1024px with FP16 half-precision).
* **Tripwire Detection:** Utilizes `cv2.pointPolygonTest` to monitor the intersection between detected lane masks and a custom 8-point sensor zone.
* **Directional Logic:**
    * The system calculates the horizontal center ($X_{mid}$) of the sensor zone.
    * If a lane line (Class 2) enters the zone:
        * **$X < X_{mid}$:** Triggers a **"Turn Right"** signal (Orange).
        * **$X > X_{mid}$:** Triggers a **"Turn Left"** signal (Yellow).

---

## HUD & Visualization Features



* **High-Opacity Zone:** A 50% alpha-blended gray overlay highlights the active "safety bubble" on the road.
* **Color-Coded Feedback:** Uses high-contrast status colors (White for clear, Orange/Yellow for alerts) to ensure driver visibility.
* **Status HUD:** A dedicated information box in the top-right corner provides real-time system feedback using `FONT_HERSHEY_DUPLEX`.

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# 1. Paths & Model Setup
WEIGHTS_PATH = '/content/drive/MyDrive/ADAS_Project/adas_final_run2/weights/best.pt'
VIDEO_PATH = '/content/drive/MyDrive/ADAS_Project/5382495-uhd_3840_2160_24fps.mp4'
OUTPUT_PATH = '/content/drive/MyDrive/ADAS_Project/ADAS_finally_System.mp4'

model = YOLO(WEIGHTS_PATH)
model.to('cuda')

# 2. Polygon Processing
raw_pts = [[440, 430], [548, 270], [710, 268], [828, 411], [712, 398], [632, 396], [553, 405], [440, 430]]
ZONE_POINTS = np.array([[int(x * 3.0), int(y * 3.0)] for x, y in raw_pts], np.int32)

# Calculate Polygon Center X for directional logic
x_coords = ZONE_POINTS[:, 0]
poly_center_x = (np.min(x_coords) + np.max(x_coords)) // 2

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(5)), (w, h))

frame_idx = 0

print(f"🚀 Processing full video @ {w}x{h} resolution...")

while True:
    ret, frame = cap.read()
    
    if not ret:
          print(f"✅ End of video stream at frame {frame_idx}")
          break


    results = model(frame, conf=0.15, imgsz=1024, half=True, verbose=False)[0]
    overlay = frame.copy()
    
    signal = "Path Clear"
    signal_color = (255, 255, 255) # White

    # --- 1. Draw HIGHER OPACITY Polygon Overlay ---
    # We use a brighter gray and will blend it with higher weight
    cv2.fillPoly(overlay, [ZONE_POINTS], (220, 220, 220))

    if results.masks is not None:
        classes = results.boxes.cls.cpu().numpy()
        masks = results.masks.xy
        
        for i, mask_data in enumerate(masks):
            cls = int(classes[i])
            pts = np.array(mask_data, dtype=np.int32)

            if cls == 0: cv2.fillPoly(overlay, [pts], (0, 0, 255))      # Car
            elif cls == 3: cv2.fillPoly(overlay, [pts], (255, 0, 0))    # Truck
            elif cls in [2, 4]: 
                cv2.fillPoly(overlay, [pts], (0, 255, 0))              # Lanes
                
                # Directional Logic for White Lines
                if cls == 2:
                    for p in pts:
                        if cv2.pointPolygonTest(ZONE_POINTS, (float(p[0]), float(p[1])), False) >= 0:
                            if p[0] < poly_center_x:
                                signal = "Turn Right"
                                signal_color = (0, 165, 255) # Orange
                            else:
                                signal = "Turn Left"
                                signal_color = (0, 255, 255) # Yellow
                            break

    # --- 2. Final HUD Assembly ---
    # Increased alpha to 0.5 for a much more visible polygon
    output_frame = cv2.addWeighted(overlay, 0.5, frame, 0.5, 0)

    # --- 3. LARGER Top Right Information Box ---
    box_w, box_h = 750, 160 # Increased width to prevent text overflow
    box_x, box_y = w - box_w - 70, 70
    
    # Darker background for better contrast
    sub_face = output_frame[box_y:box_y+box_h, box_x:box_x+box_w]
    black_rect = np.zeros(sub_face.shape, dtype=np.uint8)
    res = cv2.addWeighted(sub_face, 0.3, black_rect, 0.7, 1.0) # More opaque box
    output_frame[box_y:box_y+box_h, box_x:box_x+box_w] = res
    
    # Add Larger Signal Text
    # Font scale increased to 2.2 for maximum readability
    cv2.putText(output_frame, f"STATUS: {signal}", (box_x + 30, box_y + 100), 
                cv2.FONT_HERSHEY_DUPLEX, 2.2, signal_color, 5)

    out.write(output_frame)
    frame_idx += 1
    if frame_idx % 30 == 0: print(f"✅ Processed {frame_idx}/300")

cap.release()
out.release()
print(f"🏁 DONE! Check: {OUTPUT_PATH}")


video 1/1 (frame 1/818) d:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\5382495-uhd_3840_2160_24fps.mp4: 384x640 2 cars, 1 Driver, 1 full dashed white line, 1 yellow lane boundary, 249.8ms
video 1/1 (frame 2/818) d:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\5382495-uhd_3840_2160_24fps.mp4: 384x640 2 cars, 1 Driver, 2 full dashed white lines, 1 yellow lane boundary, 297.3ms
video 1/1 (frame 3/818) d:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\5382495-uhd_3840_2160_24fps.mp4: 384x640 2 cars, 1 Driver, 1 full dashed white line, 1 yellow lane boundary, 233.3ms
video 1/1 (frame 4/818) d:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\5382495-uhd_3840_2160_24fps.mp4: 384x640 2 cars, 1 Driver, 2 full dashed white lines, 1 yellow lane boundary, 128.8ms
video 1/1 (frame 5/818) d:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\5382495-uhd_3840_2160_24fps.mp4: 384x640 2 cars, 1 Driver, 1 full dashed white line,

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# 1. Paths & Model Setup
WEIGHTS_PATH = '/content/drive/MyDrive/ADAS_Project/adas_final_run2/weights/best.pt'
VIDEO_PATH = '/content/drive/MyDrive/ADAS_Project/5382495-uhd_3840_2160_24fps.mp4'
OUTPUT_PATH = '/content/drive/MyDrive/ADAS_Project/ADAS_finally_System.mp4'

model = YOLO(WEIGHTS_PATH)
model.to('cuda')

# 2. Polygon Processing
raw_pts = [[440, 430], [548, 270], [710, 268], [828, 411], [712, 398], [632, 396], [553, 405], [440, 430]]
ZONE_POINTS = np.array([[int(x * 3.0), int(y * 3.0)] for x, y in raw_pts], np.int32)

# Calculate Polygon Center X for directional logic
x_coords = ZONE_POINTS[:, 0]
poly_center_x = (np.min(x_coords) + np.max(x_coords)) // 2

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(5)), (w, h))

frame_idx = 0

print(f"🚀 Processing full video @ {w}x{h} resolution...")

while True:
    ret, frame = cap.read()
    
    if not ret:
          print(f"✅ End of video stream at frame {frame_idx}")
          break


    results = model(frame, conf=0.15, imgsz=1024, half=True, verbose=False)[0]
    overlay = frame.copy()
    
    signal = "Path Clear"
    signal_color = (255, 255, 255) # White

    # --- 1. Draw HIGHER OPACITY Polygon Overlay ---
    # We use a brighter gray and will blend it with higher weight
    cv2.fillPoly(overlay, [ZONE_POINTS], (220, 220, 220))

    if results.masks is not None:
        classes = results.boxes.cls.cpu().numpy()
        masks = results.masks.xy
        
        for i, mask_data in enumerate(masks):
            cls = int(classes[i])
            pts = np.array(mask_data, dtype=np.int32)

            if cls == 0: cv2.fillPoly(overlay, [pts], (0, 0, 255))      # Car
            elif cls == 3: cv2.fillPoly(overlay, [pts], (255, 0, 0))    # Truck
            elif cls in [2, 4]: 
                cv2.fillPoly(overlay, [pts], (0, 255, 0))              # Lanes
                
                # Directional Logic for White Lines
                if cls == 2:
                    for p in pts:
                        if cv2.pointPolygonTest(ZONE_POINTS, (float(p[0]), float(p[1])), False) >= 0:
                            if p[0] < poly_center_x:
                                signal = "Turn Right"
                                signal_color = (0, 165, 255) # Orange
                            else:
                                signal = "Turn Left"
                                signal_color = (0, 255, 255) # Yellow
                            break

    # --- 2. Final HUD Assembly ---
    # Increased alpha to 0.5 for a much more visible polygon
    output_frame = cv2.addWeighted(overlay, 0.5, frame, 0.5, 0)

    # --- 3. LARGER Top Right Information Box ---
    box_w, box_h = 750, 160 # Increased width to prevent text overflow
    box_x, box_y = w - box_w - 70, 70
    
    # Darker background for better contrast
    sub_face = output_frame[box_y:box_y+box_h, box_x:box_x+box_w]
    black_rect = np.zeros(sub_face.shape, dtype=np.uint8)
    res = cv2.addWeighted(sub_face, 0.3, black_rect, 0.7, 1.0) # More opaque box
    output_frame[box_y:box_y+box_h, box_x:box_x+box_w] = res
    
    # Add Larger Signal Text
    # Font scale increased to 2.2 for maximum readability
    cv2.putText(output_frame, f"STATUS: {signal}", (box_x + 30, box_y + 100), 
                cv2.FONT_HERSHEY_DUPLEX, 2.2, signal_color, 5)

    out.write(output_frame)
    frame_idx += 1
    if frame_idx % 30 == 0: print(f"✅ Processed {frame_idx}/300")

cap.release()
out.release()
print(f"🏁 DONE! Check: {OUTPUT_PATH}")

🚀 Starting inference for 300 frames...
✅ Processed 50/300 frames...
✅ Processed 100/300 frames...
✅ Processed 150/300 frames...
✅ Processed 200/300 frames...
✅ Processed 250/300 frames...
✅ Processed 300/300 frames...
🏁 Done! 300 frames saved to: D:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\ADAS_Test_300.mp4


#  Schematic Radar Mapping & HUD Implementation

The final stage of the **ADAS** project introduces **Schematic Mapping**. This technique converts complex, high-resolution visual detections into a simplified, high-contrast digital "Radar" representation, mirroring the perception displays found in modern autonomous vehicles.



---

##  Technical Implementation

The system executes a multi-layer visualization strategy to ensure driver clarity:

1.  **Geometric Filtering:** The custom 8-point polygon acts as a spatial filter. Using `cv2.pointPolygonTest`, the system ignores environmental noise and focus exclusively on objects interacting with the vehicle's immediate trajectory.
2.  **Digital Twin Projection:** Instead of standard bounding boxes, the code utilizes **Instance Segmentation Masks**. These are re-rendered onto a dedicated overlay layer to create a pixel-perfect digital twin of the road.
3.  **Alpha Blending:** Using `cv2.addWeighted` at a **0.5 ratio**, the original video is desaturated. This allows the schematic elements—Green (Lanes), Red (Cars), and Blue (Trucks)—to "pop" with maximum visibility.
---

## Advantages of Schematic Mapping

| Advantage | Description |
| :--- | :--- |
| **Cognitive Load Reduction** | Strips away visual clutter (trees, signs, sky) so the driver only sees relevant obstacles. |
| **High-Contrast Alerts** | Uses glowing status colors (Orange/Yellow) that remain visible in any lighting condition. |
| **Zero-Inference HUD** | The large-scale text box provides unambiguous instructions (**"Turn Right"**) to support the visual data. |
| **Safety Zone Transparency** | Visualizing the "safety bubble" builds user trust by showing exactly what the AI is monitoring. |

In [ ]:
# 1. Install Ultralytics
!pip install ultralytics -q

import cv2
from ultralytics import YOLO
import numpy as np
import os

# 2. Paths from your Kaggle Input
WEIGHTS_PATH = '/kaggle/input/adaslane/best (7).pt'
VIDEO_PATH = '/kaggle/input/adaslane/5382495-uhd_3840_2160_24fps.mp4'
OUTPUT_PATH = '/kaggle/working/ADAS_Kaggle_Full_Project.mp4'

# 3. Initialize Model on Kaggle GPU
model = YOLO(WEIGHTS_PATH)
model.to('cuda')

# 4. Perimeter & Radar Setup (Scaled 3x for 4K)
raw_pts = [[440, 430], [548, 270], [710, 268], [828, 411], [712, 398], [632, 396], [553, 405], [440, 430]]
ZONE_POINTS = np.array([[int(x * 3.0), int(y * 3.0)] for x, y in raw_pts], np.int32)
poly_center_x = (np.min(ZONE_POINTS[:, 0]) + np.max(ZONE_POINTS[:, 0])) // 2

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
fps = int(cap.get(5))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Get total frame count
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Enlarged Radar Dimensions
RADAR_W, RADAR_H = 600, 900
SCALE_X, SCALE_Y = RADAR_W / w, RADAR_H / h

frame_idx = 0
print(f"🚀 Kaggle Processing Started | Total Frames to Process: {total_frames}")

while cap.isOpened():
    ret, frame = cap.read()
    
    # --- REMOVED THE 300 FRAME LIMIT ---
    if not ret:
        break

    # YOLO11x Inference at 1024px
    results = model(frame, conf=0.15, imgsz=1024, half=True, verbose=False)[0]
    
    overlay = frame.copy()
    radar_view = np.zeros((RADAR_H, RADAR_W, 3), dtype=np.uint8)
    signal = "Path Clear"
    signal_color = (255, 255, 255)

    # Semi-transparent sensor zone
    cv2.fillPoly(overlay, [ZONE_POINTS], (220, 220, 220))

    if results.masks is not None:
        classes = results.boxes.cls.cpu().numpy()
        masks = results.masks.xy
        
        for i, mask_data in enumerate(masks):
            cls = int(classes[i])
            pts = np.array(mask_data, dtype=np.int32)

            # Segmentation Colors
            if cls == 0: cv2.fillPoly(overlay, [pts], (0, 0, 255))      # Car: Red
            elif cls == 3: cv2.fillPoly(overlay, [pts], (255, 0, 0))    # Truck: Blue
            elif cls in [2, 4]: cv2.fillPoly(overlay, [pts], (0, 255, 0)) # Lanes: Green
            
            # Radar Mapping
            radar_pts = (pts * [SCALE_X, SCALE_Y]).astype(np.int32)
            cv2.fillPoly(radar_view, [radar_pts], (255, 255, 255))
            
            # Steering Logic
            if cls == 2: # White Lane
                for p in pts:
                    if cv2.pointPolygonTest(ZONE_POINTS, (float(p[0]), float(p[1])), False) >= 0:
                        if p[0] < poly_center_x:
                            signal = "Turn Right"; signal_color = (0, 165, 255)
                        else:
                            signal = "Turn Left"; signal_color = (0, 255, 255)
                        break

    # Blend and Assemble HUD
    output_frame = cv2.addWeighted(overlay, 0.5, frame, 0.5, 0)

    # Radar View (Top Left)
    rx, ry = 60, 60
    cv2.rectangle(output_frame, (rx-4, ry-4), (rx+RADAR_W+4, ry+RADAR_H+4), (255, 255, 255), 3)
    car_tri = np.array([[RADAR_W//2, RADAR_H-30], [RADAR_W//2-25, RADAR_H-75], [RADAR_W//2+25, RADAR_H-75]], np.int32)
    cv2.drawContours(radar_view, [car_tri], 0, (255, 100, 0), -1)
    output_frame[ry:ry+RADAR_H, rx:rx+RADAR_W] = radar_view

    # Status Box (Top Right)
    box_w, box_h, bx, by = 750, 160, w-820, 70
    sub_face = output_frame[by:by+box_h, bx:bx+box_w]
    black_rect = np.zeros(sub_face.shape, dtype=np.uint8)
    res = cv2.addWeighted(sub_face, 0.3, black_rect, 0.7, 1.0)
    output_frame[by:by+box_h, bx:bx+box_w] = res
    cv2.putText(output_frame, f"STATUS: {signal}", (bx + 30, by + 100), 
                cv2.FONT_HERSHEY_DUPLEX, 2.2, signal_color, 5)

    out.write(output_frame)
    frame_idx += 1
    
    # Progress update every 100 frames
    if frame_idx % 100 == 0: 
        print(f"✅ Progress: {frame_idx}/{total_frames} frames processed...")

cap.release()
out.release()
print(f"🏁 PROJECT COMPLETE! Final video saved to: {OUTPUT_PATH}")

🚀 Running Inference: Class 2 (Dashes) @ 0.15 | Others @ 0.25 | No Boxes
✅ Processed 50/300
✅ Processed 100/300
✅ Processed 150/300
✅ Processed 200/300
✅ Processed 250/300
✅ Processed 300/300
🏁 Completed! File saved to D:\Desktop\Desk\Labellerr Github Projects\Use_Case_Projects\ADAS\ADAS_Custom_Threshold.mp4


---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
